In [1]:
import random,csv,math
import graphlab

In [2]:
def loadCsv(filename):
    lines = csv.reader(open(filename, "rb"))
    dataset = list(lines)
    for i in range(len(dataset)):
        dataset[i] = [int(x) for x in dataset[i]]  
        dataset[i] = [float(x) for x in dataset[i]]
    return dataset
            

def splitDataset(dataset, splitRatio):
    trainSize = int(len(dataset) * splitRatio)
    trainSet = []
    copy = list(dataset)
    while len(trainSet) < trainSize:
        index = random.randrange(len(copy))
        trainSet.append(copy.pop(index))
    return [trainSet, copy]

def normalise(dataset):
    normal=dataset
    item_inf=summarize_normal(normal)
    for i in range(len(normal)):
        for j in range(len(normal[i])-1):
            if j==0 or j==1 or j==2 or j==7 or j==11:
                normal[i][j]=(normal[i][j]-item_inf[j][0])/(item_inf[j][1])
    return normal
def separateByClass(dataset):
    separated = {}
    for i in range(len(dataset)):
        vector = dataset[i]
        if (vector[-1] not in separated):
            separated[vector[-1]] = []
        separated[vector[-1]].append(vector)
    return separated

def mean(numbers):
	return sum(numbers)/float(len(numbers))

def stdev(numbers):
	avg = mean(numbers)
	variance = sum([pow(x-avg,2) for x in numbers])/float(len(numbers)-1)
	return math.sqrt(variance)

def summarize(dataset):
    summaries = [(mean(attribute), stdev(attribute)) for attribute in zip(*dataset)]
    del summaries[-1]
    return summaries

def summarize_normal(dataset):
    meanvar = [(mean(attribute), stdev(attribute)) for attribute in zip(*dataset)]
    return meanvar

def summarizeByClass(dataset):
    separated = separateByClass(dataset)
    summaries = {}
    for classValue, instances in separated.iteritems():
        summaries[classValue] = summarize(instances)
    return summaries

def calculateProbability(x, mean, stdev):
    if stdev>0:
        exponent = math.exp(-(math.pow(x-mean,2)/(2*math.pow(stdev,2))))
        return (1 / (math.sqrt(2*math.pi) * stdev )) * exponent
    elif stdev==0:
        stdev=0.001
        exponent = math.exp(-(math.pow(x-mean,2)/(2*math.pow(stdev,2))))
        return (1 / (math.sqrt(2*math.pi) * stdev )) * exponent

def calculateClassProbabilities(summaries, inputVector):
    probabilities = {}
    for classValue, classSummaries in summaries.iteritems():
        probabilities[classValue] = 1
        for i in range(len(classSummaries)):
            if i!=3:
                mean, stdev = classSummaries[i]
                x = inputVector[i]
                k=calculateProbability(x, mean, stdev)
                probabilities[classValue] *= calculateProbability(x, mean, stdev)        
    return probabilities


def predict(summaries, inputVector):
    best_three=[]
    probabilities = calculateClassProbabilities(summaries, inputVector)
    sorteddict= sorted(probabilities.items(), key=lambda x: x[1])
    for k in range(3):
        best_three.append(sorteddict[99-k])
    #print sorteddict
    #print sorteddict[0],sorteddict[1],sorteddict[-1],sorteddict[-2],sorteddict[-3]
    #print len(sorteddict)
    return best_three
    
def getPredictions(summaries, testSet):
    predictions = []
    for i in range(len(testSet)):
        result = predict(summaries, testSet[i])
        predictions.append(result)
    for i in range(len(predictions)/1000):    
        print testSet[i]
        print predictions[i]
    return predictions
    
def getAccuracy(testSet, predictions):
	correct = 0
	for x in range(len(testSet)):
		if testSet[x][-1] == predictions[x]:
			correct += 1
	return (correct/float(len(testSet))) * 1000.0



In [3]:
def main():
    filename = 'Project/hotel_test1.csv'
    splitRatio = 0.67
    dataset = loadCsv(filename)
    #mainframe1=graphlab.SFrame('hotel_test1.csv')
    #mainframe2=graphlab.SFrame('hotel_train1.csv')
    trainingSet, testSet = splitDataset(dataset, splitRatio)
    print('Split {0} rows into train={1} and test={2} rows').format(len(dataset), len(trainingSet), len(testSet))
    refined_train_dataset=normalise(trainingSet)
    refined_test_dataset=normalise(testSet)
    

    # prepare model     
    #summaries = summarizeByClass(refined_train_dataset)
    # test model
    #predictions = getPredictions(summaries, refined_test_dataset)
    #mainframe2.show()
    #accuracy = getAccuracy(refined_test_dataset, predictions)
    #print('Accuracy: {0}%').format(accuracy)
    

In [4]:
main()

Split 93506 rows into train=62649 and test=30857 rows
